## Hinglish Text Normalizatiom

In [1]:
import json
import re
import pandas as pd
import nltk
import string

In [2]:
with open('/content/norm_words.json') as json_file:
    contractions_dict = json.load(json_file)

In [3]:
df = pd.read_csv('/content/English-Hindi-Code-Mixed-Corpus - English-Hindi code-mixed parallel corpus(1).csv')
for i in range(len(df['Sentence'])):
    df['Sentence'][i] = df['Sentence'][i].translate(str.maketrans('', '', string.punctuation))

In [4]:
output=[]
for sentence in df['Sentence']:
    sentence = sentence.lower()
    translated = [] 
    for x in sentence.split(' '): 
        t = contractions_dict.get(x, x)
        translated.append(t)
    newphrase = ' '.join(translated)
    output.append(newphrase)

## Translating to english using Google Translate API

In [5]:
!pip install googletrans==3.1.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 55 kB 3.4 MB/s 
     |████████████████████████████████| 42 kB 1.3 MB/s 
     |████████████████████████████████| 1.4 MB 38.6 MB/s 
     |████████████████████████████████| 65 kB 3.9 MB/s 
     |████████████████████████████████| 53 kB 2.1 MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16367 sha256=a6a54ff1532a0ae8daccf01d956ad02afb50e1cf6860aa3f043650cd629a8e5e
  Stored in directory: /root/.cache/pip/wheels/0c/be/fe/93a6a40ffe386e16089e44dad9018ebab9dc4cb9eb7eab65ae
Successfully built googletrans


In [6]:
from googletrans import Translator

In [7]:
#using googletranslator
translator = Translator()
translations = translator.translate(output)
english_translations = []
for translation in translations:
    english_translations.append(translation.text)

In [8]:
#to calculate the BLEU Score
def BLEUscore(originalArray,translatedArray):
    score = 0
    for i in range(len(originalArray)):
        BLEU = nltk.translate.bleu_score.sentence_bleu([originalArray[i]],translatedArray[i])
        score+=BLEU
    score = score/len(translatedArray)*100
    return score

In [9]:
print('BLEU Score: ',BLEUscore(df['English_Translation'],english_translations))

BLEU Score:  47.875108166152856


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg

## Translating into english with Hugging Face and mBART

In [10]:
!pip install transformers -U -q

     |████████████████████████████████| 4.7 MB 29.0 MB/s 
     |████████████████████████████████| 101 kB 8.8 MB/s 
     |████████████████████████████████| 6.6 MB 47.6 MB/s 


In [11]:
! pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.3 MB 25.5 MB/s 


In [12]:
!pip install google-transliteration-api

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
!pip freeze | grep transformers

transformers==4.21.1


In [14]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM 
from google.transliteration import transliterate_text #for hinglish to hindi transliteration

In [15]:
tokenizer = AutoTokenizer.from_pretrained("vasudevgupta/mbart-iitb-hin-eng") #tokenizer of the transformer

model = AutoModelForSeq2SeqLM.from_pretrained("vasudevgupta/mbart-iitb-hin-eng") #pretrained model

In [16]:
#generating pure hindi texts from hinglish corpus
hindi_texts = []
for i in output:
    hindi_texts.append(transliterate_text(i,lang_code='hi'))

In [17]:
#translation
eng = []
for i in hindi_texts:  
    model_inputs = tokenizer(i, return_tensors="pt")

    generated_tokens = model.generate(
                      **model_inputs,
                      forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"]
                      )

    translation = tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
    eng.append(translation)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1207: UserWarning: Neither `max_length` nor `max_new_tokens` have been set, `max_length` will default to 1000 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [48]:
s=eng
for i in range(len(eng)):
    s[i] = ' '.join(eng[i])

In [51]:
print('BLEU Score using MBart is: ', BLEUscore(df['English_Translation'],s))

BLEU Score using MBart is:  30.88047790468621


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg